# Build a deep denoising autoencoder
## Version 1: Getting the autoencoder started; compares two sets of hyperparameters.
Reference: https://towardsdatascience.com/implementing-an-autoencoder-in-tensorflow-2-0-5e86126e9f7 

In [ ]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import Model

class Encoder(Layer):
    def __init__(self, intermediate_dim):
        super(Encoder, self).__init__()
        self.hidden_layer = tf.keras.layers.Dense(
            units=intermediate_dim,
            activation=tf.nn.relu,
            kernel_initializer='he_uniform'
        )
        self.output_layer = tf.keras.layers.Dense(
            units=intermediate_dim,
            activation=tf.nn.sigmoid
        )

    def call(self, input_features):
        activation = self.hidden_layer(input_features)
        return self.output_layer(activation)


class Decoder(Layer):
    def __init__(self, intermediate_dim, original_dim):
        super(Decoder, self).__init__()
        self.hidden_layer = tf.keras.layers.Dense(
            units=intermediate_dim,
            activation=tf.nn.relu,
            kernel_initializer='he_uniform'
        )
        self.output_layer = tf.keras.layers.Dense(
            units=original_dim,
            activation=tf.nn.sigmoid
        )
  
    def call(self, code):
        activation = self.hidden_layer(code)
        return self.output_layer(activation)
    

class Autoencoder(Model):
    def __init__(self, intermediate_dim, original_dim):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(intermediate_dim=intermediate_dim)
        self.decoder = Decoder(intermediate_dim=intermediate_dim, original_dim=original_dim)
        self.next_encoder = Encoder(intermediate_dim = intermediate_dim)
        self.next_decoder = Decoder(intermediate_dim=intermediate_dim, original_dim=original_dim)

        
    def call(self, input_features):
        code = self.encoder(input_features)
        reconstructed = self.decoder(code)
        re_code = self.next_encoder(reconstructed)
        re_reconstructed = self.next_decoder(re_code)
        return re_reconstructed
    

def loss(model, original):
    reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(model(original), original)))
    return reconstruction_error


def train(loss, model, opt, original):
    with tf.GradientTape() as tape:
        gradients = tape.gradient(loss(model, original), model.trainable_variables)
        gradient_variables = zip(gradients, model.trainable_variables)
        opt.apply_gradients(gradient_variables)

Load MNIST

In [ ]:
import numpy as np
from tensorflow import data, constant
from tensorflow.summary import create_file_writer, record_if, scalar, image
from tensorflow import reshape
from tensorflow import optimizers

# Build a progress bar
from tensorflow.keras.utils import Progbar

batch_size = 128
learning_rate = 1e-2
epochs = 10

(training_features, _), (test_features, _) = mnist.load_data()
num_training = len(training_features)
training_features = training_features / np.max(training_features)
training_features = training_features.reshape(training_features.shape[0],
                                              training_features.shape[1] * training_features.shape[2])
training_features = training_features.astype('float32')
training_dataset = data.Dataset.from_tensor_slices(training_features)
training_dataset = training_dataset.batch(batch_size)
training_dataset = training_dataset.shuffle(training_features.shape[0])
training_dataset = training_dataset.prefetch(batch_size * 4)

progbar = Progbar(epochs*num_training/batch_size)

#autoencoder1 = Autoencoder(intermediate_dim=64, original_dim=784)
autoencoder1 = Autoencoder(intermediate_dim=42, original_dim=784)
autoencoder2 = Autoencoder(intermediate_dim=30, original_dim=784)

opt = optimizers.Adam(learning_rate=learning_rate)

writer = create_file_writer('tmp')


This block compares the output (reconstructed) images and losses of the two autoencoders defined above. The results can be viewed in TensorBoard.

In [ ]:
with writer.as_default():
    with record_if(True):
        for epoch in range(epochs):
            for step, batch_features in enumerate(training_dataset):
                train(loss, autoencoder1, opt, batch_features)
                train(loss, autoencoder2, opt, batch_features)
                loss_values1 = loss(autoencoder1, batch_features)
                loss_values2 = loss(autoencoder2, batch_features)
                original = reshape(batch_features, (batch_features.shape[0], 28, 28, 1))
                reconstructed1 = reshape(autoencoder1(constant(batch_features)), (batch_features.shape[0], 28, 28, 1))
                reconstructed2 = reshape(autoencoder2(constant(batch_features)), (batch_features.shape[0], 28, 28, 1))
                scalar('loss1', loss_values1, step=step)
                scalar('loss2', loss_values2, step=step)
                image('original', original, max_outputs=10, step=step)
                image('reconstructed1', reconstructed1, max_outputs=10, step=step)
                image('reconstructed2', reconstructed2, max_outputs=10, step=step)
                progbar.update(1+step*(epoch+1))

# Version 2: Uses KerasTuner to find better hyperparameters.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

from kerastuner.tuners import RandomSearch
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters

(X, y), (X_val, y_val) = keras.datasets.mnist.load_data()
X = X.astype('float32')/255.
X = X.reshape(X.shape[0], X.shape[1]*X.shape[2])
X_val = X_val.astype('float32')/255.
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1]*X_val.shape[2])


X_train, y_train = X[:10000], y[:10000]
X_test, y_test = X[10000:], y[10000:]


In [ ]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import Model
import tensorflow as tf

class Encoder(Layer):
    def __init__(self, intermediate_dim):
        super(Encoder, self).__init__()
        self.hidden_layer = tf.keras.layers.Dense(
            units=intermediate_dim,
            activation=tf.nn.relu,
            kernel_initializer='he_uniform'
        )
        self.output_layer = tf.keras.layers.Dense(
            units=intermediate_dim,
            activation=tf.nn.sigmoid
        )

    def call(self, input_features):
        activation = self.hidden_layer(input_features)
        return self.output_layer(activation)


class Decoder(Layer):
    def __init__(self, intermediate_dim, original_dim):
        super(Decoder, self).__init__()
        self.hidden_layer = tf.keras.layers.Dense(
            units=intermediate_dim,
            activation=tf.nn.relu,
            kernel_initializer='he_uniform'
        )
        self.output_layer = tf.keras.layers.Dense(
            units=original_dim,
            activation=tf.nn.sigmoid
        )
  
    def call(self, code):
        activation = self.hidden_layer(code)
        return self.output_layer(activation)
    

class Autoencoder(Model):
    def __init__(self, intermediate_dim, original_dim):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(intermediate_dim=intermediate_dim)
        self.decoder = Decoder(intermediate_dim=intermediate_dim, original_dim=original_dim)
        self.next_encoder = Encoder(intermediate_dim = intermediate_dim)
        self.next_decoder = Decoder(intermediate_dim=intermediate_dim, original_dim=original_dim)

        
    def call(self, input_features):
        code = self.encoder(input_features)
        reconstructed = self.decoder(code)
        re_code = self.next_encoder(reconstructed)
        re_reconstructed = self.next_decoder(re_code)
        return re_reconstructed

def loss(model, original):
    reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(model(original), original)))
    return reconstruction_error

def build_model(hp):
    model = Autoencoder(intermediate_dim = hp.Int('int_dim', 12, 72, 12), original_dim = 784)
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-3, 1e-4])),
        loss='mean_squared_error',
        metrics=['accuracy'])
    return model



Uses KerasTuner's RandomSearch to find better hyperparameters for the autoencoder.

In [ ]:
# Warning: this takes a while to run ~30min
tuner = RandomSearch(
    build_model,
    objective='accuracy',
    max_trials=20,
    executions_per_trial=3,
    directory='test',
    overwrite=True)

tuner.search_space_summary()

tuner.search(X_train,
             X_train,
             epochs=10,
             verbose = 3,
             validation_data=(X_val, X_val))

tuner.results_summary()

# lr = .0001 = 1e-3
# dim = 48




This chooses the best model, and shows some original/reconstructed image pairs.

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.fit(X_train, X_train)
best_model.summary()
best_model.save('autoencoder')

import matplotlib.pyplot as plt

originals = X_test[:10]
new = best_model.predict(originals)

for i in range(len(originals)):
    im1 = originals[i]
    nim1 = new[i]

    im1 = im1.reshape(28,28)
    nim1 = nim1.reshape(28,28)

    plt.imshow(im1, cmap='gray')
    plt.show()
    plt.imshow(nim1, cmap='gray')
    plt.show()